In [1]:
import re
import numpy as np
import pandas as pd
import sqlite3

# Data Exploration

### Load dataset `Oscar Nominees and Winners`.

In [2]:
df = pd.read_csv('academy_awards.csv', encoding='ISO-8859-1')

In [3]:
df.head()

,Year,Category,Nominee,Additional Info,Won?,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10
0,2010 (83rd),Actor -- Leading Role,Javier Bardem,Biutiful {'Uxbal'},NO,NaN,NaN,NaN,NaN,NaN,NaN
1,2010 (83rd),Actor -- Leading Role,Jeff Bridges,True Grit {'Rooster Cogburn'},NO,NaN,NaN,NaN,NaN,NaN,NaN
2,2010 (83rd),Actor -- Leading Role,Jesse Eisenberg,The Social Network {'Mark Zuckerberg'},NO,NaN,NaN,NaN,NaN,NaN,NaN
3,2010 (83rd),Actor -- Leading Role,Colin Firth,The King's Speech {'King George VI'},YES,NaN,NaN,NaN,NaN,NaN,NaN
4,2010 (83rd),Actor -- Leading Role,James Franco,127 Hours {'Aron Ralston'},NO,NaN,NaN,NaN,NaN,NaN,NaN


### There are 6 unnamed columns at the end. We want to check if any of them have valid values that we need.

In [4]:
df['Unnamed: 5'].value_counts()

*                                                                                                               7
 D.B. "Don" Keele and Mark E. Engebretson has resulted in the over 20-year dominance of constant-directivity    1
 error-prone measurements on sets. [Digital Imaging Technology]"                                                1
 resilience                                                                                                     1
 discoverer of stars                                                                                            1
Name: Unnamed: 5, dtype: int64

In [5]:
df['Unnamed: 6'].value_counts()

*                                                                   9
 flexibility and water resistance                                   1
 direct radiator bass style cinema loudspeaker systems. [Sound]"    1
 sympathetic                                                        1
Name: Unnamed: 6, dtype: int64

In [6]:
df['Unnamed: 7'].value_counts()

 while requiring no dangerous solvents. [Systems]"    1
*                                                     1
 kindly                                               1
Name: Unnamed: 7, dtype: int64

In [7]:
df['Unnamed: 8'].value_counts()

 understanding comedy genius - Mack Sennett.""    1
*                                                 1
Name: Unnamed: 8, dtype: int64

In [8]:
df['Unnamed: 9'].value_counts()

*    1
Name: Unnamed: 9, dtype: int64

In [9]:
df['Unnamed: 10'].value_counts()

*    1
Name: Unnamed: 10, dtype: int64

### `Additional Info` column contains a few different formatting styles. We are going to clean this column.

In [10]:
df['Additional Info'].value_counts()

Metro-Goldwyn-Mayer                                                                          60
Walt Disney, Producer                                                                        57
Warner Bros.                                                                                 42
John Williams                                                                                37
France                                                                                       35
Alfred Newman                                                                                34
Italy                                                                                        26
Paramount                                                                                    24
Gordon Hollingshead, Producer                                                                22
Edith Head                                                                                   22
20th Century-Fox                        

# Data Cleaning

### Some String have pattern like {'name'}, we need to extract `name`.

In [11]:
def get_movie_name(row):
    if pd.notnull(row):
        return str(row).split('{\'')[0]

In [12]:
def get_character(row):
    if pd.notnull(row):
        res = str(row).split('{\'')
        if len(res) == 2:
            return res[1].rstrip('}\'')

In [13]:
df['Movie'] = df['Additional Info'].apply(get_movie_name)

In [14]:
df['Character'] = df['Additional Info'].apply(get_character)

In [15]:
df.head()

,Year,Category,Nominee,Additional Info,Won?,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Movie,Character
0,2010 (83rd),Actor -- Leading Role,Javier Bardem,Biutiful {'Uxbal'},NO,NaN,NaN,NaN,NaN,NaN,NaN,Biutiful,Uxbal
1,2010 (83rd),Actor -- Leading Role,Jeff Bridges,True Grit {'Rooster Cogburn'},NO,NaN,NaN,NaN,NaN,NaN,NaN,True Grit,Rooster Cogburn
2,2010 (83rd),Actor -- Leading Role,Jesse Eisenberg,The Social Network {'Mark Zuckerberg'},NO,NaN,NaN,NaN,NaN,NaN,NaN,The Social Network,Mark Zuckerberg
3,2010 (83rd),Actor -- Leading Role,Colin Firth,The King's Speech {'King George VI'},YES,NaN,NaN,NaN,NaN,NaN,NaN,The King's Speech,King George VI
4,2010 (83rd),Actor -- Leading Role,James Franco,127 Hours {'Aron Ralston'},NO,NaN,NaN,NaN,NaN,NaN,NaN,127 Hours,Aron Ralston


## What's the best way to handle awards ceremonies that included movies from 2 years?

### E.g. see 1927/28 (1st) in the Year column.

In [16]:
df.iloc[10136]

Year                                                1927/28 (1st)
Category           Unique and Artistic Picture (archaic category)
Nominee                                    Paramount Famous Lasky
Additional Info                                             Chang
Won?                                                           NO
Unnamed: 5                                                    NaN
Unnamed: 6                                                    NaN
Unnamed: 7                                                    NaN
Unnamed: 8                                                    NaN
Unnamed: 9                                                    NaN
Unnamed: 10                                                   NaN
Movie                                                       Chang
Character                                                    None
Name: 10136, dtype: object

### Appending these data to the end. recording the previous row number.

In [17]:
# last 10136
total_row = df.shape[0]
print(total_row)
df = df.append( df[ (df['Year'].str.len() > 4) & (df['Year'].str.contains('/')) ], ignore_index=True)

10137


### Dealing with new data.

In [18]:
for i in range(total_row, df.shape[0]):
    search_year = re.search('(\d{2})\d{2}/(\d{2})', df.iloc[i]['Year'])
    if search_year:
        df.iloc[i]['Year'] = search_year.group(1) + search_year.group(2)

In [19]:
df[total_row:].head()

,Year,Category,Nominee,Additional Info,Won?,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Movie,Character
10137,1933,Actor -- Leading Role,Leslie Howard,Berkeley Square {'Peter Standish'} [came in 3rd],NO,NaN,NaN,NaN,NaN,NaN,NaN,Berkeley Square,Peter Standish'} [came in 3rd]
10138,1933,Actor -- Leading Role,Charles Laughton,The Private Life of Henry VIII {'Henry VIII'},YES,NaN,NaN,NaN,NaN,NaN,NaN,The Private Life of Henry VIII,Henry VIII
10139,1933,Actor -- Leading Role,Paul Muni,I Am a Fugitive from a Chain Gang {'James Alle...,NO,NaN,NaN,NaN,NaN,NaN,NaN,I Am a Fugitive from a Chain Gang,James Allen'} [came in 2nd]
10140,1933,Actress -- Leading Role,Katharine Hepburn,Morning Glory {'Eva Lovelace'},YES,NaN,NaN,NaN,NaN,NaN,NaN,Morning Glory,Eva Lovelace
10141,1933,Actress -- Leading Role,May Robson,Lady for a Day {'Apple Annie'} [came in 2nd],NO,NaN,NaN,NaN,NaN,NaN,NaN,Lady for a Day,Apple Annie'} [came in 2nd]


### Dealing with old data. Just get 

### For the `Year` column, there are many different formatting, we only need first four digits. 

In [20]:
df["Year"] = df["Year"].str[:4]

### Convert the `Year` column to `int` data type.

In [21]:
df['Year'] = df['Year'].astype(int)

In [22]:
df.head()

,Year,Category,Nominee,Additional Info,Won?,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Movie,Character
0,2010,Actor -- Leading Role,Javier Bardem,Biutiful {'Uxbal'},NO,NaN,NaN,NaN,NaN,NaN,NaN,Biutiful,Uxbal
1,2010,Actor -- Leading Role,Jeff Bridges,True Grit {'Rooster Cogburn'},NO,NaN,NaN,NaN,NaN,NaN,NaN,True Grit,Rooster Cogburn
2,2010,Actor -- Leading Role,Jesse Eisenberg,The Social Network {'Mark Zuckerberg'},NO,NaN,NaN,NaN,NaN,NaN,NaN,The Social Network,Mark Zuckerberg
3,2010,Actor -- Leading Role,Colin Firth,The King's Speech {'King George VI'},YES,NaN,NaN,NaN,NaN,NaN,NaN,The King's Speech,King George VI
4,2010,Actor -- Leading Role,James Franco,127 Hours {'Aron Ralston'},NO,NaN,NaN,NaN,NaN,NaN,NaN,127 Hours,Aron Ralston


### We only need the data after the year 2000.

In [23]:
later_than_2000 = df[df['Year'] > 2000]

In [24]:
later_than_2000.head()

,Year,Category,Nominee,Additional Info,Won?,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Movie,Character
0,2010,Actor -- Leading Role,Javier Bardem,Biutiful {'Uxbal'},NO,NaN,NaN,NaN,NaN,NaN,NaN,Biutiful,Uxbal
1,2010,Actor -- Leading Role,Jeff Bridges,True Grit {'Rooster Cogburn'},NO,NaN,NaN,NaN,NaN,NaN,NaN,True Grit,Rooster Cogburn
2,2010,Actor -- Leading Role,Jesse Eisenberg,The Social Network {'Mark Zuckerberg'},NO,NaN,NaN,NaN,NaN,NaN,NaN,The Social Network,Mark Zuckerberg
3,2010,Actor -- Leading Role,Colin Firth,The King's Speech {'King George VI'},YES,NaN,NaN,NaN,NaN,NaN,NaN,The King's Speech,King George VI
4,2010,Actor -- Leading Role,James Franco,127 Hours {'Aron Ralston'},NO,NaN,NaN,NaN,NaN,NaN,NaN,127 Hours,Aron Ralston


### There are many different types of `Category`, we only interest in four awards.

In [25]:
later_than_2000['Category'].value_counts()

Writing                                                        100
Scientific and Technical (Technical Achievement Award)          70
Best Picture                                                    60
Actor -- Leading Role                                           50
Costume Design                                                  50
Actress -- Supporting Role                                      50
Film Editing                                                    50
Art Direction                                                   50
Short Film (Live Action)                                        50
Actress -- Leading Role                                         50
Directing                                                       50
Foreign Language Film                                           50
Documentary (Feature)                                           50
Sound                                                           50
Actor -- Supporting Role                                      

In [26]:
award_categories = ['Actor -- Leading Role', 'Actor -- Supporting Role', 'Actress -- Leading Role', 'Actress -- Supporting Role']

In [27]:
nominations = later_than_2000[later_than_2000['Category'].isin(award_categories)]

In [28]:
nominations.head()

,Year,Category,Nominee,Additional Info,Won?,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Movie,Character
0,2010,Actor -- Leading Role,Javier Bardem,Biutiful {'Uxbal'},NO,NaN,NaN,NaN,NaN,NaN,NaN,Biutiful,Uxbal
1,2010,Actor -- Leading Role,Jeff Bridges,True Grit {'Rooster Cogburn'},NO,NaN,NaN,NaN,NaN,NaN,NaN,True Grit,Rooster Cogburn
2,2010,Actor -- Leading Role,Jesse Eisenberg,The Social Network {'Mark Zuckerberg'},NO,NaN,NaN,NaN,NaN,NaN,NaN,The Social Network,Mark Zuckerberg
3,2010,Actor -- Leading Role,Colin Firth,The King's Speech {'King George VI'},YES,NaN,NaN,NaN,NaN,NaN,NaN,The King's Speech,King George VI
4,2010,Actor -- Leading Role,James Franco,127 Hours {'Aron Ralston'},NO,NaN,NaN,NaN,NaN,NaN,NaN,127 Hours,Aron Ralston


### Cleaning up the `Won` column. Converting `YES` to `1`, `NO` to `0`.

In [29]:
nominations['Won?'] = nominations['Won?'].map({'YES': 1, 'NO': 0})

/Users/yuzezou/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [30]:
nominations.head()

,Year,Category,Nominee,Additional Info,Won?,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Movie,Character
0,2010,Actor -- Leading Role,Javier Bardem,Biutiful {'Uxbal'},0,NaN,NaN,NaN,NaN,NaN,NaN,Biutiful,Uxbal
1,2010,Actor -- Leading Role,Jeff Bridges,True Grit {'Rooster Cogburn'},0,NaN,NaN,NaN,NaN,NaN,NaN,True Grit,Rooster Cogburn
2,2010,Actor -- Leading Role,Jesse Eisenberg,The Social Network {'Mark Zuckerberg'},0,NaN,NaN,NaN,NaN,NaN,NaN,The Social Network,Mark Zuckerberg
3,2010,Actor -- Leading Role,Colin Firth,The King's Speech {'King George VI'},1,NaN,NaN,NaN,NaN,NaN,NaN,The King's Speech,King George VI
4,2010,Actor -- Leading Role,James Franco,127 Hours {'Aron Ralston'},0,NaN,NaN,NaN,NaN,NaN,NaN,127 Hours,Aron Ralston


### Sorting rows by `Year`.

In [31]:
nominations = nominations.sort_values(by='Year')

In [32]:
nominations.head()

,Year,Category,Nominee,Additional Info,Won?,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Movie,Character
1176,2001,Actress -- Supporting Role,Kate Winslet,Iris {'Young Iris Murdoch'},0,NaN,NaN,NaN,NaN,NaN,NaN,Iris,Young Iris Murdoch
1157,2001,Actor -- Leading Role,Russell Crowe,A Beautiful Mind {'John Nash'},0,NaN,NaN,NaN,NaN,NaN,NaN,A Beautiful Mind,John Nash
1158,2001,Actor -- Leading Role,Sean Penn,I Am Sam {'Sam Dawson'},0,NaN,NaN,NaN,NaN,NaN,NaN,I Am Sam,Sam Dawson
1159,2001,Actor -- Leading Role,Will Smith,Ali {'Muhammad Ali'},0,NaN,NaN,NaN,NaN,NaN,NaN,Ali,Muhammad Ali
1160,2001,Actor -- Leading Role,Denzel Washington,Training Day {'Alonzo'},1,NaN,NaN,NaN,NaN,NaN,NaN,Training Day,Alonzo


### Renaming `Won?` to `Won`

In [33]:
nominations = nominations.rename(columns={'Won?': 'Won'})

In [34]:
nominations = nominations.drop('Additional Info', axis=1)

### Deleting useless columns.

In [35]:
del_columns = ['Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10', ]
final_nominations = nominations.drop(del_columns, axis=1)

In [36]:
final_nominations.head()

,Year,Category,Nominee,Won,Movie,Character
1176,2001,Actress -- Supporting Role,Kate Winslet,0,Iris,Young Iris Murdoch
1157,2001,Actor -- Leading Role,Russell Crowe,0,A Beautiful Mind,John Nash
1158,2001,Actor -- Leading Role,Sean Penn,0,I Am Sam,Sam Dawson
1159,2001,Actor -- Leading Role,Will Smith,0,Ali,Muhammad Ali
1160,2001,Actor -- Leading Role,Denzel Washington,1,Training Day,Alonzo


In [37]:
final_nominations.dtypes

Year          int64
Category     object
Nominee      object
Won           int64
Movie        object
Character    object
dtype: object

# Exporting Pandas DataFrame to SQLite3 table.

### Deleting the file if the file name exist.

In [38]:
!rm -rf nominations.db

In [39]:
conn = sqlite3.connect('nominations.db')

final_nominations.to_sql(name='nominations', con=conn, index=False)

In [40]:
!ls nominations.db

nominations.db


### Printing `nominations` schema.

In [41]:
conn.execute('pragma table_info(nominations)').fetchall()

[(0, 'Year', 'INTEGER', 0, None, 0),
 (1, 'Category', 'TEXT', 0, None, 0),
 (2, 'Nominee', 'TEXT', 0, None, 0),
 (3, 'Won', 'INTEGER', 0, None, 0),
 (4, 'Movie', 'TEXT', 0, None, 0),
 (5, 'Character', 'TEXT', 0, None, 0)]

### Showing some data.

In [42]:
conn.execute('select * from nominations limit 10;').fetchall()

[(2001,
  'Actress -- Supporting Role',
  'Kate Winslet',
  0,
  'Iris ',
  'Young Iris Murdoch'),
 (2001,
  'Actor -- Leading Role',
  'Russell Crowe',
  0,
  'A Beautiful Mind ',
  'John Nash'),
 (2001, 'Actor -- Leading Role', 'Sean Penn', 0, 'I Am Sam ', 'Sam Dawson'),
 (2001, 'Actor -- Leading Role', 'Will Smith', 0, 'Ali ', 'Muhammad Ali'),
 (2001,
  'Actor -- Leading Role',
  'Denzel Washington',
  1,
  'Training Day ',
  'Alonzo'),
 (2001,
  'Actor -- Leading Role',
  'Tom Wilkinson',
  0,
  'In the Bedroom ',
  'Matt Fowler'),
 (2001,
  'Actor -- Supporting Role',
  'Jim Broadbent',
  1,
  'Iris ',
  'John Bayley'),
 (2001, 'Actor -- Supporting Role', 'Ethan Hawke', 0, 'Training Day ', 'Jake'),
 (2001,
  'Actor -- Supporting Role',
  'Ben Kingsley',
  0,
  'Sexy Beast ',
  'Don Logan'),
 (2001,
  'Actor -- Supporting Role',
  'Ian McKellen',
  0,
  'The Lord of the Rings: The Fellowship of the Ring ',
  'Gandalf')]

In [43]:
conn.close()